<a href="https://colab.research.google.com/github/mintseok/MachineLearning/blob/main/4%EC%A3%BC%EC%B0%A8/4%EC%9E%A5_%EC%8B%A0%EA%B2%BD%EB%A7%9D%ED%95%99%EC%8A%B5.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Loss Function

In [1]:
# 오차제곱합 sum of squares for error, SSE
def sum_squares_error(y, t):              #y, t는 모두 np.array 형식
  return 0.5 * np.sum((y-t)**2)           #y는 예상, t는 답

In [2]:
# 교차 엔트로피 오차 cross entropy error, CEE
def cross_entropy_error(y, t):
  delta = 1e-7                          #0.0000001 
  return -np.sum(t*np.log(y+delta))

# MiniBatch

In [3]:
!curl -O https://raw.githubusercontent.com/WegraLee/deep-learning-from-scratch/master/dataset/mnist.py

  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100  3655  100  3655    0     0  58015      0 --:--:-- --:--:-- --:--:-- 58015


In [4]:
import sys, os
import pickle
import numpy as np
from mnist import load_mnist
sys.path.append(os.pardir)

(x_train, t_train), (x_test, t_test) = \
    load_mnist(normalize=True, flatten=True, one_hot_label=False)

print(x_train.shape) #(60000, 784)=> 784는 28x28
print(t_train.shape) #(60000,)

(60000, 784)
(60000,)


In [5]:
train_size = x_train.shape[0]
batch_size = 10
batch_mask = np.random.choice(train_size, batch_size) #np.random.choice(60000, 10)은 60000 미만인 index에서 10개를 무작위로 추출
x_batch = x_train[batch_mask]
t_batch = t_train[batch_mask]
#np.random.choice(train_size, batch_size)
x_batch

array([[0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       ...,
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.]], dtype=float32)

In [6]:
#minibatch용 cross entropy loss function(for one_hot_encoding)            #어차피 minibatch로 model 만들꺼니까, 위에서 define한 함수 무시.
def cross_entropy_error(y, t):
  if y.ndim == 1:
    t = t.reshape(1, t.size)
    y = y.reshape(1, y.size)

  batch_size = y.shape[0]
  return -np.sum(t*np.log(y+1e-7)) / batch_size 

# 수치 미분

In [7]:
def numerical_diff(f, x):
  h = 1e-4                                #0.0001
  return (f(x+h) - f(x-h)) / (2*h)

# 기울기

In [8]:
def _numerical_gradient_no_batch(f, x):
    h = 1e-4 # 0.0001
    grad = np.zeros_like(x) # x와 형상이 같은 배열을 생성
    
    for idx in range(x.size):
        tmp_val = x[idx]
        
        # f(x+h) 계산
        x[idx] = float(tmp_val) + h
        fxh1 = f(x)
        
        # f(x-h) 계산
        x[idx] = tmp_val - h 
        fxh2 = f(x) 
        
        grad[idx] = (fxh1 - fxh2) / (2*h)
        x[idx] = tmp_val # 값 복원
        
    return grad

In [9]:
def numerical_gradient(f, X):
    if X.ndim == 1:
        return _numerical_gradient_no_batch(f, X)
    else:
        grad = np.zeros_like(X)
        
        for idx, x in enumerate(X):
            grad[idx] = _numerical_gradient_no_batch(f, x)
        
        return grad


'''
def numerical_gradient(f, x):
  h = 1e-4
  grad = np.zeros_like(x) # x와 형상이 같은 배열을 생성

  for idx in range(x.size):
    tmp_val = x[idx]
    # f(x+h) 계산
    x[idx] = tmp_val + h
    fxh1 = f(x)

    # f(x-h) 계산
    x[idx] = tmp_val - h
    fxh2 = f(x)

    grad[idx] = (fxh1 - fxh2) / (2*h)
    x[idx] = tmp_val # 값 복원

  return grad
  '''
  

'\ndef numerical_gradient(f, x):\n  h = 1e-4\n  grad = np.zeros_like(x) # x와 형상이 같은 배열을 생성\n\n  for idx in range(x.size):\n    tmp_val = x[idx]\n    # f(x+h) 계산\n    x[idx] = tmp_val + h\n    fxh1 = f(x)\n\n    # f(x-h) 계산\n    x[idx] = tmp_val - h\n    fxh2 = f(x)\n\n    grad[idx] = (fxh1 - fxh2) / (2*h)\n    x[idx] = tmp_val # 값 복원\n\n  return grad\n  '

# 경사 하강법

In [10]:
def gradient_descent(f, init_x, lr=0.01, step_num=100):
  x = init_x

  for i in range(step_num):
    grad = numerical_gradient(f, x)
    x -= lr * grad
  return x

# SimpleNet

In [11]:
class simpleNet:
  def __init__(self):
    self.W = np.random.randn(2, 3) # 정규분포로 초기화

  def predict(self, x):
    return np.dot(x, self.W)

  def loss(self, x, t):
    z = self.predict(x)
    y = softmax(z)
    loss = cross_entropy_error(y, t)

    return loss

# 기타 함수

In [12]:
# 시그모이드 함수 정의하기
def sigmoid(x):
  return 1 / (1+np.exp(-x))

In [13]:
# 소프트맥스 함수 정의하기
def softmax(x):
  exp_x = np.exp(x)
  sum_exp_x = np.sum(exp_x)
  return exp_x / sum_exp_x

In [14]:
def sigmoid_grad(x):
  return (1.0 - sigmoid(x)) * sigmoid(x)

# 학습 구현을 위해.. 

In [15]:
class SoftmaxWithLoss:
  def __init__(self):
    self.loss = None # 손실
    self.y = None
    self.t = None

  def forward(self, x, t):
    self.t = t 
    self.y = softmax(x)
    self.loss = cross_entropy_error(self.y, self.t)
    return self.loss

  def backward(self, dout=1):
    batch_size = self.t.shape[0]
    dx = (self.y - self.t) / batch_size
    
    return dx

In [16]:
from collections import OrderedDict

class Affine:                                              
    def __init__(self, W, b):
        self.W = W
        self.b = b
        self.x = None
        self.original_x_shape = None
        self.dW = None
        self.db = None

    def forward(self, x):
        self.x = x
        out = np.dot(self.x, self.W) + self.b

        return out

    def backward(self, dout):
        dx = np.dot(dout, self.W.T)
        self.dW = np.dot(self.x.T, dout)
        self.db = np.sum(dout, axis=0)
       
        return dx
       
class Relu:                                                  
    def __init__(self):
        self.mask = None

    def forward(self, x):
        self.mask = (x <= 0)
        out = x.copy()
        out[self.mask] = 0

        return out

    def backward(self, dout):
        dout[self.mask] = 0
        dx = dout

        return dx

# 2층 신경망 구현하기

In [17]:
from collections import OrderedDict

class TwoLayerNet:
  def __init__(self, input_size, hidden_size, output_size, weight_init_std=0.01):
    #가중치 초기화
    self.params = {}
    self.params['W1'] = weight_init_std * np.random.randn(input_size, hidden_size)
    self.params['b1'] = np.zeros(hidden_size)
    self.params['W2'] = weight_init_std * np.random.randn(hidden_size, output_size)
    self.params['b2'] = np.zeros(output_size)

    # 계층 생성
    self.layers = OrderedDict()
    self.layers['Affine1'] = Affine(self.params['W1'], self.params['b1'])
    self.layers['Relu1'] = Relu()
    self.layers['Affine2'] = Affine(self.params['W2'], self.params['b2'])

    self.lastLayer = SoftmaxWithLoss()

  def predict(self, x):
    W1, W2 = self.params['W1'], self.params['W2']
    b1, b2 = self.params['b1'], self.params['b2']

    a1 = np.dot(x, W1) + b1
    z1 = sigmoid(a1)
    a2 = np.dot(z1, W2) + b2
    y = softmax(a2)

    return y 

  # x : 입력 데이터, t : 정답 레이블
  def loss(self, x, t):
    y = self.predict(x)

    return cross_entropy_error(y, t)

  def accuracy(self, x, t):
    y = self.predict(x)
    y = np.argmax(y, axis=1) # 최대 index의 값을 반환해준다.
    t = np.argmax(t, axis=1)

    accuracy = np.sum(y==t) / float(x.shape[0])
    return accuracy
  
  def numerical_gradient(self, x, t):
    loss_W = lambda W: self.loss(x, t)

    grads = {}
    grads['W1'] = numerical_gradient(loss_W, self.params['W1'])
    grads['b1'] = numerical_gradient(loss_W, self.params['b1'])
    grads['W2'] = numerical_gradient(loss_W, self.params['W2'])
    grads['b2'] = numerical_gradient(loss_W, self.params['b2'])

    return grads

  def gradient(self, x, t):
    W1, W2 = self.params['W1'], self.params['W2']
    b1, b2 = self.params['b1'], self.params['b2']
    grads = {}
    
    batch_num = x.shape[0]
    

    a1 = np.dot(x, W1) + b1
    z1 = sigmoid(a1)
    a2 = np.dot(z1, W2) + b2
    y = softmax(a2)
    
    
    dy = (y - t) / batch_num
    grads['W2'] = np.dot(z1.T, dy)
    grads['b2'] = np.sum(dy, axis=0)
    
    da1 = np.dot(dy, W2.T)
    dz1 = sigmoid_grad(a1) * da1
    grads['W1'] = np.dot(x.T, dz1)
    grads['b1'] = np.sum(dz1, axis=0)

    return grads
    '''

    # 결과 저장
    grads = {}
    grads['W1'], grads['b1'] = self.layers['Affine1'].dW, self.layers['Affine1'].db
    grads['W2'], grads['b2'] = self.layers['Affine2'].dW, self.layers['Affine2'].db

    return grads
    '''

In [18]:
net = TwoLayerNet(input_size=784, hidden_size=100, output_size=10)
net.params['W1'].shape # (784, 100)
net.params['b1'].shape # (100,)
net.params['W2'].shape # (100, 10)
net.params['b2'].shape # (10,)

(10,)

# 학습 구현하기

In [24]:
(x_train, t_train), (x_test, t_test) = \
    load_mnist(normalize=True, one_hot_label=True)


# hyperparameters 설정
iters_num = 10000  # 반복 횟수
train_size = x_train.shape[0]
batch_size = 100      # minibatch size
learning_rate = 0.1
network = TwoLayerNet(input_size=784, hidden_size=50, output_size=10)

train_loss_list = []
train_acc_list = []
test_acc_list = []

iter_per_epoch = max(train_size / batch_size, 1)

network = TwoLayerNet(input_size=784, hidden_size=50, output_size=10)

for key in ('W1', 'b1', 'W2', 'b2'):
    network.params[key] -= learning_rate * grad[key]

loss = network.loss(x_batch, t_batch)
train_loss_list.append(loss)

for i in range(iters_num):
    # 미니배치 획득
    batch_mask = np.random.choice(train_size, batch_size)
    x_batch = x_train[batch_mask]
    t_batch = t_train[batch_mask]
    

    grad = network.gradient(x_batch, t_batch)
    
    for key in ('W1', 'b1', 'W2', 'b2'):
        network.params[key] -= learning_rate * grad[key]
    
    loss = network.loss(x_batch, t_batch)
    train_loss_list.append(loss)
    
    if i % iter_per_epoch == 0:
        train_acc = network.accuracy(x_train, t_train)
        test_acc = network.accuracy(x_test, t_test)
        train_acc_list.append(train_acc)
        test_acc_list.append(test_acc)
        print("train acc, test acc | " + str(train_acc) + ", " + str(test_acc))

KeyboardInterrupt: ignored

In [25]:
'''
(x_train, t_train), (x_test, t_test) = \
    load_mnist(normalize=True, one_hot_label=True)

train_loss_list = []

# hyperparameters 설정
iters_num = 100  # 반복 횟수
train_size = x_train.shape[0]
batch_size = 100      # minibatch size
learning_rate = 0.1
network = TwoLayerNet(input_size=784, hidden_size=50, output_size=10)

for i in range(iters_num):
  # 미니배치 획득
  batch_mask = np.random.choice(train_size, batch_size) #np.random.choice(60000, 10)은 60000 미만인 index에서 10개를 무작위로 추출
  x_batch = x_train[batch_mask]
  t_batch = t_train[batch_mask]

  # 기울기 계산
  #grad = network.numerical_gradient(x_batch, t_batch)
  grad = network.gradient(x_batch, t_batch)

  # 매개변수 갱신
  for key in ('W1', 'b1', 'W2', 'b2'):
    network.params[key] -= learning_rate * grad[key]

  # 학습 경과 기록
  loss = network.loss(x_batch, t_batch)
  train_loss_list.append(loss)
  '''

"\n(x_train, t_train), (x_test, t_test) =     load_mnist(normalize=True, one_hot_label=True)\n\ntrain_loss_list = []\n\n# hyperparameters 설정\niters_num = 100  # 반복 횟수\ntrain_size = x_train.shape[0]\nbatch_size = 100      # minibatch size\nlearning_rate = 0.1\nnetwork = TwoLayerNet(input_size=784, hidden_size=50, output_size=10)\n\nfor i in range(iters_num):\n  # 미니배치 획득\n  batch_mask = np.random.choice(train_size, batch_size) #np.random.choice(60000, 10)은 60000 미만인 index에서 10개를 무작위로 추출\n  x_batch = x_train[batch_mask]\n  t_batch = t_train[batch_mask]\n\n  # 기울기 계산\n  #grad = network.numerical_gradient(x_batch, t_batch)\n  grad = network.gradient(x_batch, t_batch)\n\n  # 매개변수 갱신\n  for key in ('W1', 'b1', 'W2', 'b2'):\n    network.params[key] -= learning_rate * grad[key]\n\n  # 학습 경과 기록\n  loss = network.loss(x_batch, t_batch)\n  train_loss_list.append(loss)\n  "